In [5]:
from pyspark import SparkConf, SparkContext
conf1 = SparkConf().setMaster("local").setAppName("spark1")
sc = SparkContext(conf = conf1)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark1, master=local) created by __init__ at <ipython-input-1-1be344c730dd>:3 

# 1 数据类型

## 1.1 RDD

In [6]:
intRDD = sc.parallelize([1,3,5,7])

In [7]:
strRdd =  sc.parallelize(["1","3","5","7"])

In [8]:
RDD = sc.parallelize([["李四",15],["王五",19],["张三",18]])

In [9]:
RDDtuple = sc.parallelize([("李四",15),("王五",19),("张三",18)])

In [10]:
int2RDD = intRDD.map(lambda x:x**2)
int2RDD.collect()

[1, 9, 25, 49]

In [11]:
int2RDD.first()

1

## 1.2DataFrame

spark.createDataFrame(数据,shema=所有列)

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("spark-python").getOrCreate()

#列1 二维列表数据,schema是一维列表
data=[("李四",15),["王五",19],("张三",18)]
columns=("name","age")
df = spark.createDataFrame(data,schema=columns)#默认数据类型
df.printSchema() #查看数据类型
print(df.collect())#查看数,以Row的形式返回
df.show()#查看数据
df.describe().show()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

[Row(name='李四', age=15), Row(name='王五', age=19), Row(name='张三', age=18)]
+----+---+
|name|age|
+----+---+
|李四| 15|
|王五| 19|
|张三| 18|
+----+---+

+-------+----+------------------+
|summary|name|               age|
+-------+----+------------------+
|  count|   3|                 3|
|   mean|null|17.333333333333332|
| stddev|null|2.0816659994661326|
|    min|张三|                15|
|    max|王五|                19|
+-------+----+------------------+



In [15]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,LongType

In [16]:
#列2 二维列表数据,schema是StructType
data = [["李四", 15], ["王五", 19], ["张三", 18]]
columns = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

#df = spark.createDataFrame(RDD,schema=columns)
df = spark.createDataFrame(data, schema=columns)

df.printSchema()  #查看数据类型
print(df.collect())  #查看数,以Row的形式返回
df.show()  #查看数据
df.describe().show()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

[Row(name='李四', age=15), Row(name='王五', age=19), Row(name='张三', age=18)]
+----+---+
|name|age|
+----+---+
|李四| 15|
|王五| 19|
|张三| 18|
+----+---+

+-------+----+------------------+
|summary|name|               age|
+-------+----+------------------+
|  count|   3|                 3|
|   mean|null|17.333333333333332|
| stddev|null|2.0816659994661326|
|    min|张三|                15|
|    max|王五|                19|
+-------+----+------------------+



In [17]:
df.select("name").show()

+----+
|name|
+----+
|李四|
|王五|
|张三|
+----+



In [18]:
from  pyspark.sql import Row
#列3 rdd(row对象)
rdd1 = sc.textFile("hdfs://127.0.0.1:9000/sparkml-data/file0.txt")
rdd2 = rdd1.map(lambda x: x.split(","))
rdd3 = rdd2.map(lambda x: Row(name=x[0],age=int(x[1])))

print(rdd3.collect())

df = spark.createDataFrame(rdd3)
df.printSchema() #查看数据类型

df.show()#查看数据

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.net.ConnectException: Call From USER-20180910KA/172.16.74.106 to 127.0.0.1:9000 failed on connection exception: java.net.ConnectException: Connection refused: no further information; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:792)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:732)
	at org.apache.hadoop.ipc.Client.call(Client.java:1479)
	at org.apache.hadoop.ipc.Client.call(Client.java:1412)
	at org.apache.hadoop.ipc.Protob

$$
    \frac{\sum(x_i-\hat{x}^2)}{n}
$$

In [19]:
pwd

'E:\\Program\\PycharmProjects\\study\\pyspark'

In [20]:
#列4 从文件中读取,结果是DataFram
df1 = spark.read.csv("file:/root/pyspark_code/pyspark/data/2.txt",header=True)#所有类型是字符串
df2 = spark.read.option("header",True).json("file:/root/pyspark_code/pyspark/data/1.json")
df3 = spark.read.option("header","true").csv("file:/root/pyspark_code/pyspark/data/2.txt")
df4 = spark.read.csv("file:/root/pyspark_code/pyspark/data/3.txt")#默认读取没有头

AnalysisException: 'Path does not exist: file:/root/pyspark_code/pyspark/data/2.txt;'

In [64]:
df1.show()
df2.show()
df3.show()
df4.show()

+------+---+
|  name|age|
+------+---+
|zhang3| 18|
|   li4| 19|
| wang5| 20|
+------+---+

+---+----+
|age|name|
+---+----+
| 19|  张三|
| 18|  李四|
| 20|  王五|
+---+----+

+------+---+
|  name|age|
+------+---+
|zhang3| 18|
|   li4| 19|
| wang5| 20|
+------+---+

+------+---+
|   _c0|_c1|
+------+---+
|zhang3| 18|
|   li4| 19|
| wang5| 20|
+------+---+



In [65]:
df1.printSchema()
df2.printSchema()
df3.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)



In [66]:
df1 = spark.read.csv("file:/root/pyspark_code/pyspark/data/2.txt",header=True,inferSchema=True)#inferSchema自动探索数据类型

In [68]:
df1.printSchema()
df1.show()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+------+---+
|  name|age|
+------+---+
|zhang3| 18|
|   li4| 19|
| wang5| 20|
+------+---+



In [71]:
df5 = spark.read.json("file:/root/pyspark_code/pyspark/data/1.json")
df = spark.read.load("file:/root/pyspark_code/pyspark/data/2.txt",format="csv",header=True,sep=",",inferSchema=True)

In [73]:
df5.show()
df5.printSchema()
df.show()
df.printSchema()

+---+----+
|age|name|
+---+----+
| 19|  张三|
| 18|  李四|
| 20|  王五|
+---+----+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+------+---+
|  name|age|
+------+---+
|zhang3| 18|
|   li4| 19|
| wang5| 20|
+------+---+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



In [79]:
#列5 写文件
df.write.csv("file:/root/pyspark_code/pyspark/data/df")#默认不写头

#df.write.csv("data_dir",header=True)
df.write.csv("hdfs://127.0.0.1:9000//sparkml-data/df",header=True,mode="overwrite")#append为追加

In [80]:
df.write.parquet("file:/root/pyspark_code/pyspark/data/1.parquet")

In [81]:
spark.read.parquet("file:/root/pyspark_code/pyspark/data/1.parquet").show()

+------+---+
|  name|age|
+------+---+
|zhang3| 18|
|   li4| 19|
| wang5| 20|
+------+---+



In [81]:
#与df.write.parquet("1.parquet") 相同
df.write.save("2.parquet",format="parquet")

In [84]:
#例6:查询df的api函数或者spark.sql

In [85]:
#选择某列,加上条件
df.select("age").filter(df.age<20).show()

+---+
|age|
+---+
| 18|
| 19|
+---+



In [21]:
#创建并替换-------------------------------仅仅创建,不替换,如有重名,报错
df.createOrReplaceTempView("student")==df.registerTempTable("student")

True

In [22]:
spark.sql("select age from student where age < 20").show()

+---+
|age|
+---+
| 15|
| 19|
| 18|
+---+



In [26]:
#例8 pandas的DataFrame和spark的DataFrame的互相转换
import pandas as pd
pandas_DF=pd.DataFrame([
    ["zhang3",18],
    ['lisi',19],
    ["wang5",20]
],columns=["name","age"],index=["a","b","c"]
)
spark_df=spark.createDataFrame(pandas_DF)

spark_df.show()
pandas_DF

+------+---+
|  name|age|
+------+---+
|zhang3| 18|
|  lisi| 19|
| wang5| 20|
+------+---+



,name,age
a,zhang3,18
b,lisi,19
c,wang5,20


In [27]:
#spark转换成pandas
spark_df.toPandas()

,name,age
0,zhang3,18
1,lisi,19
2,wang5,20


In [28]:
#例9 rdd和spark.dataframe的互相转换
rdd = df.rdd #dataframe->rdd
rdd.collect()


[Row(name='李四', age=15), Row(name='王五', age=19), Row(name='张三', age=18)]

In [31]:
#rdd->dataframe
df2 = spark.createDataFrame(rdd)
df3 = rdd.toDF()
df2.show()
df3.show()

+----+---+
|name|age|
+----+---+
|李四| 15|
|王五| 19|
|张三| 18|
+----+---+

+----+---+
|name|age|
+----+---+
|李四| 15|
|王五| 19|
|张三| 18|
+----+---+



# 特征工程的几个组件
# 1.字符串转成数字StringIndexer

In [32]:
from pyspark.ml.feature import StringIndexer
data=[
    ["李四",15,"高中"],
    ["王五",19,"本科"],
    ["张三",18,"硕士"],
    ["赵6",20,"本科"]
]
columns=["name","age","学历"]
df = spark.createDataFrame(data,schema=columns)
df.show()

+----+---+----+
|name|age|学历|
+----+---+----+
|李四| 15|高中|
|王五| 19|本科|
|张三| 18|硕士|
| 赵6| 20|本科|
+----+---+----+



In [35]:

obj = StringIndexer(inputCol="学历",outputCol="学历2")#返回transfomer对象
result=obj.fit(df)#返回transfomer对象,fit训练,并储存中间结果
result.labels

['本科', '硕士', '高中']

In [36]:
df2 = result.transform(df)
df2.show()
df2

+----+---+----+-----+
|name|age|学历|学历2|
+----+---+----+-----+
|李四| 15|高中|  2.0|
|王五| 19|本科|  0.0|
|张三| 18|硕士|  1.0|
| 赵6| 20|本科|  0.0|
+----+---+----+-----+



DataFrame[name: string, age: bigint, 学历: string, 学历2: double]

In [93]:
# 2.独热编码OneHotEncoder
from pyspark.ml.feature import OneHotEncoder
obj = OneHotEncoder(inputCol="学历2",outputCol="学历3",dropLast=False)
df3 = obj.transform(df2)
df3.show()#学历3 第二个表示[n] 第几个是1 其他都是0

+----+---+---+---+-------------+
|name|age| 学历|学历2|          学历3|
+----+---+---+---+-------------+
|  李四| 15| 高中|1.0|(3,[1],[1.0])|
|  王五| 19| 本科|0.0|(3,[0],[1.0])|
|  张三| 18| 硕士|2.0|(3,[2],[1.0])|
|  赵6| 20| 本科|0.0|(3,[0],[1.0])|
+----+---+---+---+-------------+

